In [1082]:
#import necessary packages
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
from lxml import html
import requests
import csv

# Importing the School Codes

The way I got the PhD codes for the schools where my faculty received was downloading a current version of the school code Google Sheets sheet. I then converted it into a .csv using excel and then imported that excel file into my script. Saving each value into a dictionary allowed me to then find edge cases and either insert them if they were new schools into both my csv and the Google Sheet but also if they were different renditions of the school I could write an edge case and throw it into the dictionary.

In [1083]:
#Imports a csv containing all of the school-ids, will attach with 
with open('school-id.csv', 'rU') as csvfile:
    reader = csv.reader(csvfile, dialect='excel')
    schoolDict = {rows[-1]:rows[0] for rows in reader}
    schoolDictInts = dict((str(k),int(v)) for k,v in schoolDict.iteritems())
#Edge case where Texas A&M is 'Texas A & M'
schoolDictInts["Texas A & M University"] = 32
#Edge case where Cal is 'Berkeley'
schoolDictInts["Berkeley"] = 1
#Edge case where Cal is 'U. C. Berkeley'
schoolDictInts["U. C. Berkeley"] = 1
#Edge case where Cal is 'University of California at Berkeley'
schoolDictInts["University of California at Berkeley"] = 1
#Edge case where Virginia Tech is 'Virginia Polytechnic Institute and State University'
schoolDictInts["Virginia Polytechnic Institute and State University"] = 406
#Edge case where University of Illinois, Urbana Champaign is 'University of Illinois'
schoolDictInts["University of Illinois"] = 117
#Edge case where Cal is "University of California"
schoolDictInts["University of California"] = 1
#Edge case where UC Irvine is '"University of California, Irvine"'
schoolDictInts['"University of California, Irvine"'] = 7

# University of Washington - Tacoma Business School

The UW Tacoma Business School faculty directory was an actual directory with a nice table displaying all of the information. Out of all three schools I scraped, this was probably the easiest to work with.

In [1084]:
urlUWT = "https://www.tacoma.uw.edu/milgard/directory/faculty" #url
uwtPage = urllib2.urlopen(urlUWT) #open page
uwtSoup = BeautifulSoup(uwtPage.read(), "html.parser") #parse page

In [1085]:
uwtPeople = uwtSoup.find_all("tr") #People were stored in tr elements so it was easy to find

In [1086]:
uwtPeopleText = []
for each in uwtPeople:
    currText = each.get_text().split("\n")
    currText = [x for x in currText if (x)]
    currText = [x for x in currText if not (x.isspace())]
    if not (currText[0] == "Name"):
        uwtPeopleText.append(currText)

In [1087]:
uwtFname = []
uwtLname = []
uwtDept = []
uwtPhd = []
uwtID = []
uwtPhdID = []

for x in uwtPeopleText:
    #There were a few cases where faculty had mbas or other phd like degrees like edh or jd
    if ("Ph.D." in x[len(x)-1]):
        name = x[0].split(",")
        uwtFname.append(name[1].strip())
        uwtLname.append(name[0].strip())
        #There were a few edge cases for where they worked. Most had their department but these four had titles
        if name[0] == "Sivadas":
            uwtDept.append("Master of Business Administration")
        elif name[0] == "Feroz":
            uwtDept.append("Accouting")
        elif name[0] == "Norman":
            uwtDept.append("Business Administration")
        elif name[0] == "Noronha":
            uwtDept.append("Business Administration")
        #Everyone else had their department
        else:
            uwtDept.append(x[2])
        phd = x[len(x)-1].split(",")
        #Not stripping initially 
        uwtPhd.append(phd[1].strip())
        uwtID.append("369")
for each in uwtPhd:
    if schoolDictInts[each]:
        uwtPhdID.append(int(schoolDictInts[each]))
    else:
        print each #lets me see what's missing from the dictionary

In [1088]:
uwtData = pd.DataFrame({'First Name': uwtFname, 'Last Name': uwtLname, 'Department of faculty position': uwtDept, 'School of PhD': uwtPhd, 'PhD ID': uwtPhdID, 'University ID': uwtID, "School of faculty position":"University of Washington - Tacoma"})
uwtData = uwtData[['First Name', 'Last Name', 'School of PhD', 'PhD ID', 'School of faculty position', 'University ID', 'Department of faculty position']]
print uwtData.shape
uwtData

(24, 7)


,First Name,Last Name,School of PhD,PhD ID,School of faculty position,University ID,Department of faculty position
0,Zoe,Barsness,Northwestern University,235,University of Washington - Tacoma,369,Management
1,Margo,Bergman,University of Houston,280,University of Washington - Tacoma,369,Finance
2,Daniel,Bryan,University of Oregon,146,University of Washington - Tacoma,369,Accounting
3,Zhiyan,Cao,Yale University,21,University of Washington - Tacoma,369,Accounting
4,Sergio,Davalos,University of Arizona,229,University of Washington - Tacoma,369,Finance
5,Haluk,Demirkan,University of Florida,220,University of Washington - Tacoma,369,Data Analytics
6,Marion,Eberly,University of Washington,24,University of Washington - Tacoma,369,Management
7,Ehsan H.,Feroz,University of Chicago,122,University of Washington - Tacoma,369,Accouting
8,Shalini,Jain,University of Washington,24,University of Washington - Tacoma,369,Management
9,Rupinder,Jindal,INSEAD,380,University of Washington - Tacoma,369,Marketing


# Virginia Commonwealth University - Marketing Department

The VCU Marketing Department provided their PhD information on their fulltime faculty page. Not the easiest to collect but not the worst. I only had to write one edge case!

In [1089]:
urlVCUM = "http://business.vcu.edu/departments-and-centers/marketing/faculty-and-staff/full-time-faculty--staff/" #url
vcumPage = urllib2.urlopen(urlVCUM) #open page
vcumSoup = BeautifulSoup(vcumPage.read(), "html.parser") #parse page

In [1090]:
vcumPeople = vcumSoup.find_all("tr") #Info was stored in tr elements but contained other unecessary text as well
vcumPeopleText = []
for each in vcumPeople:
    currText = each.get_text().split("\n")
    currText = [x for x in currText if (x)] #A lot of the lines were empty so I wrote this code to remove empty ones
    currText = [x for x in currText if not (x.isspace())]
    if currText and currText[0].startswith("Dr. "):
        vcumPeopleText.append(currText)
#print vcumPeopleText[1][2] - note this is where education is

In [1091]:
#Create lists that will be imported to a dataframe
vcumFname = []
vcumLname = []
vcumPhd = []
vcumDept = []
vcuid = []
vcuPhdId = []

for x in vcumPeopleText:
    nameText = x[0].split("Dr.")[1] #Each person had a Ph.D. so their title was always Dr.
    nameText = nameText.split(",")[0]
    nameText = nameText.split(" ") #cleanly gives you first and last names
    vcumFname.append(nameText[1].strip())
    vcumLname.append(nameText[len(nameText)-1].strip())
    phdText = x[2] #Formatted the same way so I could tell x[2] always had the necessary phd info
    
    #Slough was the one edge case where their PhD wasn't the same as everyone else's since they mentioned he had two
    #from VCU
    if "Slough" in nameText[len(nameText)-1]:
        phdText = "Virginia Commonwealth University"
    
    #PhD was written a few ways so I needed to write cases for each situation
    if "PhD" in phdText:
        temp = phdText.split("PhD")
        for x in temp:
                if ("University" in x):
                    phdText = x.strip()
    elif "Ph.D." in phdText:
        temp = phdText.split("Ph.D.")
        for x in temp:
            if ("University" in x):
                phdText = x.strip()
    else:
        temp = phdText.split("Doctor of Philosophy")
        for x in temp:
            if ("University" in x):
                phdText = x.strip()
    
    #These are for the text right before the School name, there were only 2 cases, the and from
    if("the" in phdText):
        splitTemp = phdText.split("the")
        for x in splitTemp:
            if ("University" in x):
                phdText = x.strip()
                
    if("from" in phdText):
        splitTemp = phdText.split("from")
        for x in splitTemp:
            if ("University" in x):
                phdText = x.strip()
    
    #These cases removed the text at the end or if there were random comas in front of the school name.            
    if("." in phdText):
        splitTemp = phdText.split(".")
        for x in splitTemp:
            if ("University" in x):
                phdText = x.strip()
                
    if("," in phdText):
        splitTemp = phdText.split(",")
        for x in splitTemp:
            if ("University" in x):
                phdText = x.strip()
                
    if("at " in phdText):
        splitTemp = phdText.split("at")
        for x in splitTemp:
            if ("University" in x):
                phdText = x.strip()
    vcumPhd.append(phdText)
    vcumDept.append("Marketing")
    vcuid.append("60")

#iterates through the PhDs and finds associated IDs
for each in vcumPhd:
    if schoolDictInts[each]:
        vcuPhdId.append(int(schoolDictInts[each]))
    else:
        print each #lets me see what's missing from the dictionary

In [1092]:
vcumData = pd.DataFrame({'First Name': vcumFname, 'Last Name': vcumLname, 'Department of faculty position': vcumDept, 'School of PhD': vcumPhd, 'PhD ID': vcuPhdId, 'University ID': vcuid, "School of faculty position":"Virginia Commonwealth University"})
vcumData = vcumData[['First Name', 'Last Name', 'School of PhD', 'PhD ID', 'School of faculty position', 'University ID', 'Department of faculty position']]
vcumData

,First Name,Last Name,School of PhD,PhD ID,School of faculty position,University ID,Department of faculty position
0,Brian,Brown,Georgia State University,407,Virginia Commonwealth University,60,Marketing
1,Deborah,Cowles,Arizona State University,23,Virginia Commonwealth University,60,Marketing
2,Jodie,Ferguson,Georgia State University,407,Virginia Commonwealth University,60,Marketing
3,Frank,Franzak,University of Maryland,191,Virginia Commonwealth University,60,Marketing
4,Haeran,Jae,University of Kentucky,201,Virginia Commonwealth University,60,Marketing
5,Kenneth,Kahn,Virginia Polytechnic Institute and State Unive...,406,Virginia Commonwealth University,60,Marketing
6,Suzanne,Makarem,Temple University,303,Virginia Commonwealth University,60,Marketing
7,Wayne,Slough,Virginia Commonwealth University,60,Virginia Commonwealth University,60,Marketing
8,Van,Wood,University of Oregon,146,Virginia Commonwealth University,60,Marketing


# Cal State, Los Angeles Business School

This is where the bulk of my content came from. All of their departments minus one had the information provided in somewhat easy ways to scrape. The departments I struggled with the most were Management and Economics.

In [1093]:
#create soup for IS page
urlCSt = "http://www.calstatela.edu/business/is/faculty"
cstPage = urllib2.urlopen(urlCSt) #open page
cstSoup = BeautifulSoup(cstPage.read(), "html.parser") #parse page

#create soup for accouting page
urlACCTGCst = "http://www.calstatela.edu/business/accounting/fulltime-faculty-directory"
cstACPage = urllib2.urlopen(urlACCTGCst)
cstACSoup = BeautifulSoup(cstACPage.read(), "html.parser")

#create soup for econ page
urlECONCst = "http://www.calstatela.edu/business/econ/full-time-faculty-and-office-hours"
cstECONPage = urllib2.urlopen(urlECONCst)
cstECONSoup = BeautifulSoup(cstECONPage.read(), "html.parser")

#create soup for FIN page
urlFINCst = "http://www.calstatela.edu/business/fin/faculty-directory"
cstFINPage = urllib2.urlopen(urlFINCst)
cstFINSoup = BeautifulSoup(cstFINPage.read(), "html.parser")

#create soup for MKT page
urlMKTCst = "http://www.calstatela.edu/business/mkt/faculty-and-areas-specialization"
cstMKPage = urllib2.urlopen(urlMKTCst)
cstMKSoup = BeautifulSoup(cstMKPage.read(), "html.parser")

#create soup for MGMT page
urlMGMTCst = "http://www.calstatela.edu/business/news/management"
cstMGPage = urllib2.urlopen(urlMGMTCst)
cstMGSoup = BeautifulSoup(cstMGPage.read(), "html.parser")

In [1094]:
#Find elements that have people information
cstPeople = cstSoup.find_all("p", {"style" : "margin: 0in 0in 0pt;"})
cstACPeople = cstACSoup.find_all("p")
cstECONPeople = cstECONSoup.find_all("span")
cstFINPeople = cstFINSoup.find_all("p")
cstMKTPeople = cstMKSoup.find_all("p")
cstMGPeople = cstMGSoup.find_all("div", {"class" : "field field-name-field-staff-member field-type-field-collection field-label-hidden"})

#Create lists that will store information to be put in dataframe
cstfName = []
cstlName = []
cstPhd = []
cstDept = []
cstID = []
cstPhdId = []

#for IS department
for x in cstPeople:
    tempText = x.get_text()
    #a few faculty members do not have phds
    if ("Ph.D." in tempText):
        tempText = tempText.split("(")
        for xx in tempText:
            #splits education text to find phd
            if ("Ph.D." in xx):
                phdText = xx
        #their names were always first
        nameText = tempText[0].strip()
        tempText = nameText.split(" ")
        cstfName.append(tempText[0])
        cstlName.append(tempText[-1])
        #phds were always in ()
        tempText = phdText.split(")")
        for y in tempText:
            if ("Ph.D." in y):
                phdText = y.split(".D.")[-1]
                #this splits up the text so that only the school name remains
                tempPhdText = phdText.split(",")
                for each in tempPhdText:
                    if ("University" in each) or ("Academy" in each):
                        phdText = each.strip()
        cstPhd.append(phdText)
        cstDept.append("Information Systems")
        cstID.append(3)

#For Accouting Department
for x in cstACPeople:
    tempText = x.get_text()
    #Some had CPAs or non PhD degrees
    if ("Ph.D." in tempText):
        nameTextAC = x.find_all("strong")[0].get_text()
        getNameText = nameTextAC.split(",")
        cstfName.append(getNameText[-1].strip())
        cstlName.append(getNameText[0].strip())
        #the Phd text was stored right after a br tag
        for br in x.findAll('br'):
            nextThing = br.nextSibling
            if ("Ph.D.," in nextThing):
                #this extracts the college name
                tempPhd = nextThing.split(",")
                nextThing = tempPhd[-1]
                tempPhd = nextThing.split(";")
                nextThing = tempPhd[0]
                #makes sure its a string
                cstPhd.append(str(nextThing.strip()))
        cstDept.append("Accounting")
        cstID.append(3)

#For Economics
for x in cstECONPeople:
    tempText = x.get_text()
    infoText = tempText.split(",")
    for each in infoText:
        if ("University" in each) or ("UC " in each):
            #There was one case where the footer was in the same tag.
            if (not "2016 Trustees of" in each):
                phdText = each.split(";")[0]
                #If statements are for edge cases where the text was connected to what they got their PhD in
                if("Econ" in phdText):
                    phdText = phdText.split("Econ")[0]
                if("Macro" in phdText):
                    phdText = phdText.split("Macro")[0]
                if("Urban" in phdText):
                    phdText = phdText.split("Urban")[0]
                if("Public" in phdText):
                    phdText = phdText.split("Public")[0]
                if("Applied" in phdText):
                    phdText = phdText.split("Applied")[0]
                if("Labor" in phdText):
                    phdText = phdText.split("Labor")[0]
                if("Stat" in phdText):
                    phdText = phdText.split("Statis")[0]
                if("Political" in phdText):
                    phdText = phdText.split("Political")[0]
                if ("," in phdText):
                    cstPhd.append('"' + phdText.strip() + '"')
                else:
                    #made sure to attach a string for comparison with dictionary
                    cstPhd.append(str(phdText.strip()))
                cstDept.append("Economics")
                cstID.append(3)
    
    #name info was stored in the strong tag
    name = x.find_all("strong")
    if name:
        #to extract the fname and lname I needed to remove ',' and extra spaces
        if (not name[0].get_text() == "Retired Active Faculty"):
            tempText = name[0].get_text().split(",")
            nameTempText = tempText[0]
            tempText = nameTempText.split(" ")
            cstfName.append(tempText[0].strip())
            cstlName.append(tempText[-1].strip())

#For Finance
#probably one of the more clean departments
for x in cstFINPeople:
    tempText = x.get_text()
    #takes into account for non phd degrees
    if ("Ph.D." in tempText):
        name = tempText.split("(Ph.D.,")[0].strip()
        phd = tempText.split("(Ph.D.,")[-1]
        tempText = name.split(" ")
        cstfName.append(tempText[0].strip())
        cstlName.append(tempText[-1].strip())
        tempText = phd.split(") ")
        #this is where I began to see colleges with ',' in their name like University of Illinois, blah
        #created a case to take that into account
        if ("," in tempText[0]):
            cstPhd.append('"' + str(tempText[0].strip()) + '"')
        else:    
            cstPhd.append(str(tempText[0].strip()))
        cstDept.append("Finance")
        cstID.append(3)
        
#For Marketing
#also much cleaner
for x in cstMKTPeople:
    tempText = x.get_text()
    if ("Ph.D." in tempText):
        name = tempText.split("Ph.D.,")[0].strip()
        name = name.split("(")[0]
        #print name
        phd = tempText.split("Ph.D.,")[-1]
        #print phd
        tempText = name.split(" ")
        cstfName.append(tempText[0].strip())
        cstlName.append(tempText[-1].strip())
        tempText = phd.split(") ")
        #same ',' case
        if ("," in tempText[0]):
            cstPhd.append('"' + tempText[0].strip() + '"')
        else:    
            cstPhd.append(tempText[0].strip())
        cstDept.append("Marketing")

#For Management Department
#This is much messier so there will be a few edge cases for getting the PhD
for x in cstMGPeople:
    if "Ph.D." in x.get_text():
        tempText = x.get_text().split("\n")
        #eliminates blank or lines with only spaces
        tempText = [x for x in tempText if (x)]
        tempText = [x for x in tempText if not (x.isspace())]
        #their names were always first
        name = tempText[0]
        name = name.strip()
        nameTemp = name.split(" ")
        cstfName.append(nameTemp[0].strip())
        cstlName.append(nameTemp[-1].strip())
        cstDept.append("Management")
        #gets phd school name, this was difficult
        #edge cases where PhD school is in a paragraph randomly
        if (nameTemp[-1].strip() == "Pelly"):
            cstPhd.append("EMLYON Business School")
        elif (nameTemp[-1].strip() == "O'Brien"):
            cstPhd.append("University of Southern California")
        elif (nameTemp[-1].strip() == "Yoon"):
            cstPhd.append("George Washington University")
        elif (nameTemp[-1].strip() == "Canavati"):
            cstPhd.append("University of Missouri-Kansas City")
        else:
            phdText = ''
            #seperates out where the school name is
            for each in tempText:
                if ("Ph.D." in each):
                        phdText = each.strip()
            tempText = phdText.split("Ph.D.")
            for each in tempText:
                if("University" in each):
                    phdText = each
            phdText = phdText.split(",")[-1]
            phdText = phdText.split(";")[0]
            cstPhd.append(str(phdText.strip()))

#Find PhD School IDs
for each in cstPhd:
    if each in schoolDictInts:
        cstPhdId.append(int(schoolDictInts[each]))
    else:
        print each #troubleshoots 

In [1095]:
cstData = pd.DataFrame({'First Name': cstfName, 'Last Name': cstlName, 'Department of faculty position': cstDept, 'School of PhD': cstPhd, 'PhD ID': cstPhdId, 'University ID': cstID, "School of faculty position":'"Cal State, Los Angeles"', 'University ID': 3})
cstData = cstData[['First Name', 'Last Name', 'School of PhD', 'PhD ID', 'School of faculty position', 'University ID', 'Department of faculty position']]
print cstData.shape
cstData

(68, 7)


,First Name,Last Name,School of PhD,PhD ID,School of faculty position,University ID,Department of faculty position
0,Shilpa,Balan,University of Mississippi,291,"""Cal State, Los Angeles""",3,Information Systems
1,Nandakumar,Ganesan,University of Birmingham,409,"""Cal State, Los Angeles""",3,Information Systems
2,David,Gadish,University of Guelph,299,"""Cal State, Los Angeles""",3,Information Systems
3,Adam,Huarng,University of Memphis,404,"""Cal State, Los Angeles""",3,Information Systems
4,Parviz,Partow-Navid,University of Texas,49,"""Cal State, Los Angeles""",3,Information Systems
5,Jose,Perez-Carballo,New York University,72,"""Cal State, Los Angeles""",3,Information Systems
6,Ludwig,Slusky,Academy of Municipal Economy,395,"""Cal State, Los Angeles""",3,Information Systems
7,Ming,Wang,Southern Illinois University,410,"""Cal State, Los Angeles""",3,Information Systems
8,Jongwook,Woo,University of Southern California,4,"""Cal State, Los Angeles""",3,Information Systems
9,Song,Xing,George Mason University,411,"""Cal State, Los Angeles""",3,Information Systems


In [1096]:
#Append the 3 different dataframes together now that they are complete
x = cstData.append(vcumData)
fullData = x.append(uwtData)
#add in the unrequired empty rows
fullData["Department of PhD"] = ""
fullData["Year of PhD"] = ""
fullData["Year faculty started"] = ""
fullData["job title"] = ""
#After appending all of the different dataframes together, I reset the index so it looks nicer.
fullData = fullData.reset_index(drop=True)
#Reorganize columns to fit requirement
fullData = fullData[['First Name', 'Last Name', 'School of PhD', 'PhD ID', 'Department of PhD', 'Year of PhD', 'School of faculty position', 'University ID', 'Department of faculty position', 'Year faculty started', 'job title']]

In [1098]:
print fullData.shape
#Sends dataframe to a .csv file I specified, don't want the index so set it to false
fullData.to_csv("output.csv", sep='\t', index=False, encoding='utf-8')
fullData

(101, 11)


,First Name,Last Name,School of PhD,PhD ID,Department of PhD,Year of PhD,School of faculty position,University ID,Department of faculty position,Year faculty started,job title
0,Shilpa,Balan,University of Mississippi,291,,,"""Cal State, Los Angeles""",3,Information Systems,,
1,Nandakumar,Ganesan,University of Birmingham,409,,,"""Cal State, Los Angeles""",3,Information Systems,,
2,David,Gadish,University of Guelph,299,,,"""Cal State, Los Angeles""",3,Information Systems,,
3,Adam,Huarng,University of Memphis,404,,,"""Cal State, Los Angeles""",3,Information Systems,,
4,Parviz,Partow-Navid,University of Texas,49,,,"""Cal State, Los Angeles""",3,Information Systems,,
5,Jose,Perez-Carballo,New York University,72,,,"""Cal State, Los Angeles""",3,Information Systems,,
6,Ludwig,Slusky,Academy of Municipal Economy,395,,,"""Cal State, Los Angeles""",3,Information Systems,,
7,Ming,Wang,Southern Illinois University,410,,,"""Cal State, Los Angeles""",3,Information Systems,,
8,Jongwook,Woo,University of Southern California,4,,,"""Cal State, Los Angeles""",3,Information Systems,,
9,Song,Xing,George Mason University,411,,,"""Cal State, Los Angeles""",3,Information Systems,,


# Kansas University Business School

Although they were already used, I didn't realize it until Monday so I spent the weekend scraping it. Didn't include it in my final number but I thought I would keep the work I did to show why some of the other work might have been done in a faster manner since this didn't work out.

In [ ]:
#I worked a lot on the KU Biz School (6ish hours)
#Throwing the code in to show why some of my other stuff could have seemed rushed.
urlKU = "https://business.ku.edu/faculty"
#Create soup object for KU Biz School
kuPage = urllib2.urlopen(urlKU) #open page
kuSoup = BeautifulSoup(kuPage.read(), "html.parser") #parse page
kuPeople = kuSoup.find_all("div", { "class" : "facultyrow"})

#Retrieve the text
kuPeopleText = []
for each in kuPeople:
    currText = each.get_text()
    kuPeopleText.append(currText)

kuPeopleClean = []
for each in peopleText:
    splitText = each.split("\n")
    splitText = [x for x in splitText if (x)]
    splitText = [x for x in splitText if not (x.isspace())]
    if "Professor" in splitText[1]:
        faculty = []
        faculty.append(splitText[0].strip())
        faculty.append(splitText[1].strip())
        for x in splitText:
            if ("Ph.D." in x) or ("PhD" in x) or ("Doctor of Philosophy" in x):
                faculty.append(x.strip())
        if (len(faculty) > 2):
            kuPeopleClean.append(faculty)

kuFname = []
kuLname = []
kuAcademics = []
kuPhd = []
for x in kuPeopleClean:
    name = x[0].split()
    nFname = name[0]
    nLname = name[len(name)-1]
    kuFname.append(nFname)
    kuLname.append(nLname)
    kuPosition.append(x[1])
    phdTemp = ''
    if x[0] == "Paul Koch":
        kuPhd.append("Michigan State") #fringe case where position is phd advisor
    elif x[0] == "Neeli Bendapudi":
        kuPhd.append("University of Kansas") #fringe case where education section is a paragraph
    elif x[0] == "Niki A. den Nieuwenboer":
        kuPhd.append("Erasmus University") #fringe case where univeristy is formatted differently due to it being international
    elif x[0] == "John Sweeney":
        kuPhd.append("University of Missouri") #fringe case where University of Missouri is difficult to extract
    elif x[0] == "Timothy L. Shaftel":
        kuPhd.append("Carnegie Mellon University")
    elif x[0] == "Robert (Bob) DeYoung":
        kuPhd.append("University of Wisconsin-Madison") #case where — was present and was not representable by unicode
    elif x[0] == "Lei Li":
        kuPhd.append("Boston College")
    elif x[0] == "William L. Fuerst":
        kuPhd.append("Texas Tech University")
    else:
        phdTemp = x[2]
        #print phdTemp
        if phdTemp.startswith("PhD Uni") or (phdTemp.startswith("PhD Texas") or phdTemp.startswith("PhD Fin") or phdTemp.startswith("PhD C") or phdTemp.startswith("PhD W")):
            split = phdTemp.split("D") #case where we need to split a the "D" in PhD
            phdTemp = split[1].strip()
        elif (phdTemp.startswith("Ph.D.,")) or (phdTemp.startswith("Ph.D. in")) or (phdTemp.startswith("Doctor of Philosophy,")) or (phdTemp.startswith("Ph.D. (")) or (phdTemp.startswith("PhD,")):
            split = phdTemp.split(",") #cases where we need to split at a ','
            for i in split:
                if ("University" in i):
                    phdTemp = i.strip()
        elif (phdTemp.startswith("Ph.D. ")):
            split = phdTemp.split(".") #case where we need to split at the "." in Ph.D.
            for i in split:
                if ("University" in i):
                    phdTemp = i.strip()

        #eliminates "," content if still present
        if ("," in phdTemp):
            splitTemp = phdTemp.split(",")
            for x in splitTemp:
                if ("University" in x):
                    phdTemp = x.strip()
                
        #eliminates date if in PhD string still
        if("1" in phdTemp):
            splitTemp = phdTemp.split("1")
            for x in splitTemp:
                if ("University" in x):
                    phdTemp = x.strip()
        
        #eliminates ()
        splitTemp = phdTemp.split("(")
        for x in splitTemp:
            if ("University" in x):
                phdTemp = x.strip()
        splitTemp = phdTemp.split(")")
        for x in splitTemp:
            if ("University" in x):
                phdTemp = x.strip()
        
        #removes 'The' if it's at the front of the school i.e. The Ohio State University
        if(phdTemp.startswith("The ")):
            splitTemp = phdTemp.split("The")
            for x in splitTemp:
                if ("University" in x):
                    phdTemp = x.strip()
        #removes any unecessary " - " leading to additional unecessary info like which department they goth their PhD 
        if (" - " in phdTemp):
            splitTemp = phdTemp.split("-")
            for x in splitTemp:
                if ("University" in x):
                    phdTemp = x.strip()
                    
        kuPhd.append(phdTemp)
kuData = pd.DataFrame({'First Name': kuFname, 'Last Name': kuLname, 'Department of faculty position': "Business School", 'School of PhD': kuPhd, 'University ID': "374", "School of faculty position":"University of Kansas", 'University ID': "374"})
kuData = kuData[['First Name', 'Last Name', 'School of PhD', 'University ID', 'School of faculty position', 'University ID', 'Department of faculty position']]
kuData